In [185]:
input_dir = '/Users/augustineigboke/Documents/Image-Segmentation/6dicomscans'
output_dir = '/Users/augustineigboke/Documents/Image-Segmentation/LiTs24/dicom_groups'
nifti_output_dir = '/Users/augustineigboke/Documents/Image-Segmentation/LiTs24/nifti_files'

In [190]:
from glob import glob
import shutil
import os
import sys
import dicom2nifti

In [186]:
class ImageSementationUtils:
    def __init__(self, input_dir, output_dir):
        self.__input_dir = input_dir
        self.__output_dir = output_dir
        self.__dataset_folders = glob(self.__input_dir + '/*')
        self.__group_size = self.get_group_size()

    # Sets the group size to a non-zero folder with the least files
    def get_group_size(self):
        min = sys.maxsize
        for folderume in glob(self.__input_dir + '/*'):
            l = len(glob(folderume + '/*'))
            min = l if l != 0 and l < min else min
        return min

    def prepare_dicom(self):
        group_folders = self.create_output_dirs(self.__group_size)
        if len(group_folders) > 0:
            self.move_folderumes_to_output_folders(group_folders)
        else:
            print('Empty Folder(s)')
        

    # Partitions the original data into groups of equal sizes
    def create_output_dirs(self, group_size):
        group_folders = []
        if group_size <= 0:
            return group_folders
        #Loop through all the folders in the original dataset
        for folder in self.__dataset_folders:
            folder_name = os.path.basename(os.path.normpath(folder))
            # Get the number of groups that can be formed from each folder
            num_groups = int(len(glob(folder + '/*')) / group_size)
            # Create folders according to the number of groups calculated above
            for i in range(num_groups):
                group_folder = os.path.join(self.__output_dir, folder_name + '_' + str(i))
                # Remove folder if already existing
                if os.path.exists(group_folder):
                    shutil.rmtree(group_folder)
                os.mkdir(group_folder)
                group_folders.append(group_folder)
        return group_folders

    # Moves files in defined sizes from the original folderume files or patients to the created groups
    def move_folderumes_to_output_folders(self, group_folders):
        current_folder = 0
        for folder in self.__dataset_folders:
            num_folder_groups = int(len(glob(folder + '/*')) / self.__group_size)
            files_in_folder = sorted(glob(folder + '/*'))
            
            start = 0
            for i in range(num_folder_groups):
                self.move_files_to_output_dir(files_in_folder[start : start + self.__group_size], group_folders[current_folder])
                start = start + self.__group_size
                current_folder = current_folder + 1

    # Moves chunk of files to the specified folder
    def move_files_to_output_dir(self, files, group_folder):
        group_size = self.get_group_size()
        for i, file in enumerate(files):
            if i > self.__group_size:
                break
            shutil.move(file, group_folder)

    # Converts dicom files to nifti
    def dicom2nifti(self, nitfi_ouput_dir):
        for dicom_group in glob(self.__output_dir + '/*'):
            dicom_group_name = os.path.basename(os.path.normpath(dicom_group))
            dicom2nifti.dicom_series_to_nifti(dicom_group, os.path.join(nitfi_ouput_dir, dicom_group_name + '.nii.gz'))

    # Removes nifti file with dimension less than 3 (background, foreground, mask)
    def remove_empty_niftis(self, nitfi_ouput_dir):
        for file in glob(nitfi_ouput_dir + '/*'):
            nifti_file = nib.load(file)
            fdata = nifti_file.get_fdata()
            np_unique = np.unique(fdata)
            if len(np_unique) < 3:
                print(f"Deleting empty nifty file :: {file}")
                os.remove(file)

        

In [ ]:
img_seg_utils = ImageSementationUtils(input_dir, output_dir)
group_size = img_seg_utils.get_group_size()
print(group_size)
img_seg_utils.prepare_dicom()


In [189]:
img_seg_utils = ImageSementationUtils(input_dir, output_dir)
img_seg_utils.dicom2nifti(nifti_output_dir)

In [ ]:
img_seg_utils = ImageSementationUtils(input_dir, output_dir)
img_seg_utils.remove_empty_niftis(nifti_output_dir)